# **Training on PBMC**
<a target="_blank" href="https://colab.research.google.com/github/raphaelrubrice/scVAE_mva2025/blob/raph/PBMC_experiments.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## **Colab setup**

In [1]:
!git clone https://github.com/raphaelrubrice/scVAE_mva2025.git
%cd scVAE_mva2025
!python -m pip install -r requirements.txt
!git checkout raph

Cloning into 'scVAE_mva2025'...
remote: Enumerating objects: 544, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 544 (delta 136), reused 161 (delta 97), pack-reused 319 (from 1)
Receiving objects: 100% (544/544), 2.42 MiB | 7.01 MiB/s, done.
Resolving deltas: 100% (314/314), done.
/content/scVAE_mva2025
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.1/284.1 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 143.5 MB/s eta 0:00:00
Branch 'raph' set up to track remote branch 'raph' from 'origin'.
Switc

### Test Downloads

In [2]:
from data_pipeline.src.downloader import run_downloads

run_downloads()


Saving extracted PBMC datasets to: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw

⇩ Downloading and extracting to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD34


CD34: 100%|██████████| 38.0M/38.0M [00:00<00:00, 144MB/s]


✓ Extracted to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD34
✓ Finished CD34
------------------------------------------------------------
⇩ Downloading and extracting to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD19_B


CD19_B: 100%|██████████| 18.0M/18.0M [00:00<00:00, 138MB/s]


✓ Extracted to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD19_B
✓ Finished CD19_B
------------------------------------------------------------
⇩ Downloading and extracting to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD56_NK


...

✓ Extracted to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD4_CD45RO
✓ Finished CD4_CD45RO
------------------------------------------------------------
⇩ Downloading and extracting to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD8


CD8: 100%|██████████| 20.0M/20.0M [00:00<00:00, 124MB/s]


✓ Extracted to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD8
✓ Finished CD8
------------------------------------------------------------
⇩ Downloading and extracting to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD8_CD45RA


CD8_CD45RA: 100%|██████████| 20.9M/20.9M [00:00<00:00, 81.5MB/s]


✓ Extracted to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD8_CD45RA
✓ Finished CD8_CD45RA
------------------------------------------------------------
All PBMC datasets ready in: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw


In [3]:
from data_pipeline.src.config import DATASETS
from data_pipeline.src.load_anndata import load_anndata
from pathlib import Path
import os

folder = Path("data_pipeline/data/pbmc_raw/CD4_CD45RO")
meta = DATASETS["CD4_CD45RO"]

adata = load_anndata(folder, meta)
adata

Loading 10x data from: data_pipeline/data/pbmc_raw/CD4_CD45RO/filtered_matrices_mex/hg19
✓ Loaded 10224 cells × 32738 genes from CD4_CD45RO


AnnData object with n_obs × n_vars = 10224 × 32738
    obs: 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', 'cell_type_lvl4', 'dataset', 'cell_type_lvl1__code', 'cell_type_lvl2__code', 'cell_type_lvl3__code', 'cell_type_lvl4__code'
    var: 'gene_ids'

In [4]:
from data_pipeline.src.combine import run_combine

combined, collection = run_combine(
    do_write_shards=True,
    write_combined=True,
    harmonize_var=False
)

Loading all PBMC AnnData objects...
📦 Loading dataset: CD34
Loading 10x data from: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD34/filtered_matrices_mex/hg19
✓ Loaded 9232 cells × 32738 genes from CD34
📦 Loading dataset: CD19_B
Loading 10x data from: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD19_B/filtered_matrices_mex/hg19
✓ Loaded 10085 cells × 32738 genes from CD19_B
📦 Loading dataset: CD56_NK
Loading 10x data from: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD56_NK/filtered_matrices_mex/hg19
✓ Loaded 8385 cells × 32738 genes from CD56_NK
📦 Loading dataset: CD4_helper
Loading 10x data from: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD4_helper/filtered_matrices_mex/hg19
✓ Loaded 11213 cells × 32738 genes from CD4_helper
📦 Loading dataset: CD4_CD25
Loading 10x data from: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD4_CD25/filtered_matrices_mex/hg19
✓ Loaded 10263 cells × 32738 genes from CD4_CD25
📦 Loading dataset: CD4_CD45RA_CD25neg
Loading 

/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1796: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


✓ Combined AnnData written to /content/scVAE_mva2025/data_pipeline/data/pbmc_processed/pbmc_combined.h5ad
✓ Label maps saved to /content/scVAE_mva2025/data_pipeline/data/pbmc_processed/label_maps.json
Building in‑memory AnnCollection...


/usr/local/lib/python3.12/dist-packages/legacy_api_wrap/__init__.py:88: UserWarning: Observation names are not unique.
  return fn(*args_all, **kw)


✓ AnnCollection constructed from 9 shards.

Combine stage complete!



In [5]:
import scanpy as sc
adata_combined = sc.read_h5ad("data_pipeline/data/pbmc_processed/pbmc_combined.h5ad")
adata_combined

/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1796: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 92043 × 32738
    obs: 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', 'cell_type_lvl4', 'dataset', 'cell_type_lvl1__code', 'cell_type_lvl2__code', 'cell_type_lvl3__code', 'cell_type_lvl4__code'
    var: 'gene_ids'
    uns: 'label_maps'

To avoid some warnings down stream

In [ ]:
import torch.utils.data.dataloader as tdl

_original_del = tdl._MultiProcessingDataLoaderIter.__del__

def _quiet_del(self):
    try:
        _original_del(self)
    except AssertionError as e:
        if "can only test a child process" in str(e):
            # swallow this specific noisy assertion
            pass
        else:
            raise

tdl._MultiProcessingDataLoaderIter.__del__ = _quiet_del

In [6]:
from data_pipeline.src.dataloader import build_cv_dataloaders

N_GENES = 500 # Authors used 5000 most variation, we can take 10% of that
kept_idx, folds, test_loader = build_cv_dataloaders(
    shard_dir="data_pipeline/data/pbmc_processed/shards",
    label_maps_path="data_pipeline/data/pbmc_processed/label_maps.json",
    batch_size=256,
    one_hot=True,
    num_workers=os.cpu_count() // 2,
    pin_m=True,
    filter_genes=True,
    max_genes=N_GENES,
    downsample=True, # take 10% of the dataset
    frac=0.1
)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (ipython-input-1605742747.py, line 9)

# **Cross-Validation of architectures**

In [ ]:
import torch
from tqdm.auto import tqdm
from mixture_vae.run_handling import run_cv
from mixture_vae.viz import plot_loss_components, plot_latent

In [ ]:
# Problem setup
input_dim = N_GENES # genes
hidden_dim = 100 # hidden neurons per layer
n_components = 9 # 9 clusters are assumed
latent_dim = 100 # dimension latent space
n_layers = 2 # number of encoding and decoding layers

LR = 1e-3
EPOCHS = 50
beta_kl = 1
warmup = int(0.4*EPOCHS)

## **Cross-Validated scVAE**

In [ ]:
from mixture_vae.mvae import MixtureVAE

In [ ]:
# model + priorlatent + prior_input + posterior latent (we will always use uniform prior categorical ?)
save_path = "./scVAE_Normal_NB_Normal.ckpt"
run_tag = "scVAE_Normal_NB_Normal"

In [ ]:
config_mvae = {
        # Run name and model
        "run_tag": run_tag,
        "model_type": "MixtureVAE",

        # priors and posteriors
        "prior_latent_dist": "Normal",
        "prior_input_dist": "NegativeBinomial",
        "posterior_latent_dist": "Normal",
        "prior_categorical_dist": "Uniform",

        # architecture params
        "input_dim": input_dim,
        "hidden_dim": hidden_dim,
        "latent_dim": latent_dim,
        "n_components": n_components,
        "n_layers": n_layers,

        # training params
        "lr": 1e-3,
        "epochs": EPOCHS,
        "beta_kl": beta_kl,
        "warmup": warmup,
        "save_path": save_path
    }

In [ ]:
cv_mvae, mvae_metrics = run_cv(config_mvae, folds, test_loader,
                                plot_latent_space=True, progress_bar=True, show_loss_every=5)

In [ ]:
mvae_metrics.to_csv(f"./{run_tag}/{run_tag}_metrics.csv")
mvae_metrics

## **Cross-Validated independent Mixture of Mixtures**

In [ ]:
from mixture_vae.mvae import ind_MoMVAE

In [ ]:
# model + priorlatent + prior_input + posterior latent (we will always use uniform prior categorical ?)
save_path = "./Ind_MoM_Normal_NB_Normal.ckpt"
run_tag = "Ind_MoM_Normal_NB_Normal"

In [ ]:
config_indmom = {
        # Run name and model
        "run_tag": run_tag,
        "model_type": "ind_MoMVAE",

        # priors and posteriors
        "prior_latent_dist": "Normal",
        "prior_input_dist": "NegativeBinomial",
        "posterior_latent_dist": "Normal",
        "prior_categorical_dist": "Uniform",

        # architecture params
        "input_dim": input_dim,
        "hidden_dim": hidden_dim,
        "latent_dim": latent_dim,
        "hierarchy_components": [2, 3, 5, 9],
        "n_layers": n_layers,

        # training params
        "lr": 1e-3,
        "epochs": EPOCHS,
        "beta_kl": beta_kl,
        "warmup": warmup,
        "save_path": save_path
    }

In [ ]:
cv_indmom, indmom_metrics = run_cv(config_indmom, folds, test_loader,
                                plot_latent_space=True, progress_bar=True, show_loss_every=5)

In [ ]:
indmom_metrics.to_csv(f"./{run_tag}/{run_tag}_metrics.csv")
indmom_metrics

## **Cross-Validated Mixture of Mixtures**

In [ ]:
from mixture_vae.mvae import MoMixVAE

In [ ]:
# model + priorlatent + prior_input + posterior latent (we will always use uniform prior categorical ?)
save_path = "./MoMix_Normal_NB_Normal.ckpt"
run_tag = "MoMix_Normal_NB_Normal"

In [ ]:
config_momix = {
        # Run name and model
        "run_tag": run_tag,
        "model_type": "MoMixVAE",

        # priors and posteriors
        "prior_latent_dist": "Normal",
        "prior_input_dist": "NegativeBinomial",
        "posterior_latent_dist": "Normal",
        "prior_categorical_dist": "Uniform",

        # architecture params
        "input_dim": input_dim,
        "hidden_dim": hidden_dim,
        "latent_dim": latent_dim,
        "hierarchy_components": [2, 3, 5, 9],
        "n_layers": n_layers,

        # training params
        "lr": 1e-3,
        "epochs": EPOCHS,
        "beta_kl": beta_kl,
        "warmup": warmup,
        "save_path": save_path
    }

In [ ]:
cv_momix, momix_metrics = run_cv(config_momix, folds, test_loader,
                                plot_latent_space=True, progress_bar=True, show_loss_every=5)

In [ ]:
momix_metrics.to_csv(f"./{run_tag}/{run_tag}_metrics.csv")
momix_metrics

# **Figure 1: Generative and Clustering performances**

In [ ]:
from mixture_vae.figure1 import plot_figure1

In [ ]:
for i in range(1,5):
    plot_figure1([mvae_metrics, indmom_metrics, momix_metrics], radj_level=f'lvl.{i}', save_path=f"./Figure1_lvl{i}.pdf")